In [1]:
#172.18.0.4
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
import os

In [3]:
CATALOG_URI = "http://nessie:19120/api/v1"  # Nessie Server URI
WAREHOUSE = "s3://warehouse/"               # Minio Address to Write to
STORAGE_URI = "http://172.18.0.4:9000"     # Minio IP address from docker inspect


In [53]:
spark.stop()

In [4]:
conf = (
    pyspark.SparkConf()
        .setAppName('billing')
    
    # .set("spark.executor.instances", "3")  # 3 Executors
    # .set("spark.executor.cores", "3")  # 3 Cores per executor
    # .set("spark.executor.memory", "3g")  # 3GB RAM per executor
    # .set("spark.driver.memory", "4g")  # Ensure driver has enough memory
    # .set("spark.sql.shuffle.partitions", "200")  # Tune for large joins
    # .set("spark.default.parallelism", "9")  # Optimize parallel processing
    # .set("spark.executor.instances", "3")  # 3 Executors
    # .set("spark.executor.cores", "2")  # 3 Cores per executor
    # .set("spark.executor.memory", "6g")  # 3GB RAM per executor
    # .set("spark.driver.memory", "6g")  # Ensure driver has enough memory
    # .set("spark.sql.shuffle.partitions", "200")  # Tune for large joins
    # .set("spark.default.parallelism", "200")  # Optimize parallel processing
    # .set("spark.memory.fraction", "0.7")  #
    # .set("spark.executor.memoryOverhead", "1024") 
#     .set("spark.dynamicAllocation.enabled", "true")
# .set("spark.shuffle.service.enabled", "true")
        .set("spark.sql.debug.maxToStringFields", "100000")
        .set('spark.jars', '/opt/spark/workjars/iceberg-spark-runtime-3.5_2.12-1.5.0.jar,/opt/spark/workjars/nessie-spark-extensions-3.5_2.12-0.77.1.jar,/opt/spark/workjars/bundle-2.24.8.jar,/opt/spark/workjars/url-connection-client-2.24.8.jar')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set("spark.executor.memory", "2g")
        .set("spark.driver.memory", "2g")
        .set("spark.executor.memoryOverhead", "512m")
        .set("spark.sql.shuffle.partitions", "64")
        .set("spark.shuffle.spill", "true")
        .set("spark.shuffle.memoryFraction", "0.4")
#         .set("spark.executor.instances", "2")  # Use 2 executors
#         .set("spark.executor.cores", "2")  # 2 cores per executor
#         .set("spark.executor.memory", "5g")  # 5GB per executor (10GB total)
#         .set("spark.driver.memory", "2g")  # 2GB for the driver
#         .set("spark.memory.fraction", "0.6")  # Lower memory fraction to leave room for overhead
#         .set("spark.executor.memoryOverhead", "1g")  # 1GB overhead per executor
#         .set("spark.sql.shuffle.partitions", "100")  # Adjust based on data size
#         .set("spark.default.parallelism", "100")  # Adjust parallelism
#         .set("spark.dynamicAllocation.enabled", "true")  # Enable dynamic allocation
#         .set("spark.shuffle.service.enabled", "true")  # Required for dynamic allocation
#         .set("spark.sql.debug.maxToStringFields", "100000")
#         .set('spark.jars', '/opt/spark/workjars/iceberg-spark-runtime-3.5_2.12-1.5.0.jar,/opt/spark/workjars/nessie-spark-extensions-3.5_2.12-0.77.1.jar,/opt/spark/workjars/bundle-2.24.8.jar,/opt/spark/workjars/url-connection-client-2.24.8.jar')
#         .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
#         .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
#         .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
#         .set('spark.sql.catalog.nessie.ref', 'main')
#         .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
#         .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
#         .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
#         .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
#         .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
#     .set("spark.shuffle.service.enabled", "true")  # Enable external shuffle service
# .set("spark.shuffle.service.port", "7337")  # Default shuffle service port
)


In [5]:
spark = SparkSession.builder.master("local[*]").config(conf=conf).getOrCreate()
print("Spark Session Started")

25/03/23 12:17:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/23 12:17:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Session Started


In [15]:
result = spark.sql("SHOW TABLES IN nessie").collect()

In [16]:
result

[Row(namespace='EnergyConsumptionFeederwise', tableName='EnergyConsumptionFeederwise_data_raw', isTemporary=False),
 Row(namespace='ageing', tableName='ageing_data_raw', isTemporary=False),
 Row(namespace='billing', tableName='billing_data_raw', isTemporary=False),
 Row(namespace='crm', tableName='crm_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery', tableName='fact_billing_and_recovery_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery2', tableName='fact_billing_and_recovery_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery3', tableName='fact_billing_and_recovery_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery4', tableName='fact_billing_and_recovery_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery5', tableName='fact_billing_and_recovery_data_raw', isTemporary=False),
 Row(namespace='fact_billing_and_recovery_final1', tableName='fact_billing_and_recovery_data_raw', isT

In [17]:
for a in results:
    
if 'test' in [namespace for namespace in result]:
    print("yes")
else:
    print("no")
    

no


In [8]:
print(spark.conf.get("spark.jars.packages", "No Iceberg packages found"))
print(spark.conf.get("spark.sql.catalog.nessie", "No Nessie catalog found"))

No Iceberg packages found
org.apache.iceberg.spark.SparkCatalog


In [10]:
spark.stop()

In [9]:
# spark = SparkSession.builder.master("spark://spark:7077").getOrCreate()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestExecutors") \
    .master("spark://spark:7077") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()


25/02/22 12:58:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [18]:

num_executors = spark._jsc.sc().getExecutorMemoryStatus().size() - 1  # Exclude driver

# Get number of cores per executor
num_cores = spark.sparkContext.defaultParallelism

# Get memory per executor
executor_memory = spark.conf.get("spark.executor.memory", "Not Set")

print(f"Cluster Info:")
print(f"  - Executors: {num_executors}")
print(f"  - Cores per executor: {num_cores}")
print(f"  - Executor Memory: {executor_memory}")

Cluster Info:
  - Executors: 1
  - Cores per executor: 12
  - Executor Memory: Not Set


In [17]:
df = spark.range(1000).rdd.getNumPartitions()
print(df)


12


In [168]:
print(spark._jsc.sc().getExecutorMemoryStatus())


Map(034b7b88bc5d:37659 -> (455501414,455501414))


In [ ]:
spark = SparkSession.builder \
    .appName("TestApp") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,software.amazon.awssdk:bundle:2.24.8,software.amazon.awssdk:url-connection-client:2.24.8") \
    .getOrCreate()
print("Spark Session Started")

In [6]:
ls "/mnt/HabibData/"

Ageing_202301.csv*


In [51]:
csv = "/mnt/HabibData/Ageing_202303.csv"

In [52]:

# Read the CSV file into a DataFrame
df = spark.read \
    .option("delimiter", "|") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(csv)

# df.show()
# df.printSchema()
df = df.repartition(200)  # Match shuffle partitions
# spark = SparkSession.builder.config(conf=conf).getOrCreate()
# print("Spark Session Started")
df.writeTo("nessie.ageing.ageing_data_raw").createOrReplace()

In [9]:
df = df.repartition(200)  # Match shuffle partitions

OMS

In [45]:
# spark.sql("DROP table nessie.oms.oms_data_raw").show()
# spark.sql("DROP schema nessie.oms").show()
spark.sql("CREATE NAMESPACE nessie.oms").show()

++
||
++
++



In [41]:
csv = "/mnt/HabibData/filtered_fault_data.csv"
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(csv)


In [42]:
# df.select('Time').limit(1).show()
df.limit(1).show(truncate=False)

+----------+-------------------+----------+-----+-----+-------------------+-------------+-------------------+---------------+------+--------------+---------+-----+-----------+--------------+---+--------+------+--------------------+----------+-------+-------+----+---+--------+------+----------+-----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+
|Createdon |punchCloseAt       |OutageType|saifi|saidi|consumer_count_fdr5|outageSubType|initialoffreason   |htclosingreason|fdr_Id|fdr_name      |grid_name|Relay|isemergency|israintripping|TAT|duration|dts_Id|consumer_count_fdr18|datetime  |tempmax|tempmin|temp|dew|humidity|precip|precipprob|precipcover|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|
+----------+-------------------+----------+-----+-----+-------------------+-------------+-------------------+-

25/02/27 17:52:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk
 Schema: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr5, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr18, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solare

In [46]:
# spark = SparkSession.builder.config(conf=conf).getOrCreate()
# print("Spark Session Started")
df.writeTo("nessie.oms.oms_data_raw").createOrReplace()

25/02/27 17:53:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk
 Schema: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr5, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr18, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solare

CRM

In [47]:
csv = "/mnt/HabibData/CRM_202301-202501.csv"
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(csv)


In [ ]:
# spark.sql("DROP table nessie.crm.crm_data_raw").show()
# spark.sql("DROP schema nessie.crm").show()
spark.sql("CREATE NAMESPACE nessie.crm").show()

In [10]:
# df.select('Time').limit(1).show()
df.limit(1).show(truncate=False)

+--------------------+--------+----------------+---------------------+-------------------+---------------+----------------+--------+------------+-------------+-------------+-------------+------+---+-------+----+-----+------+----------------+----------------+----------------------+---------------+-----------+---+------+---------+-------+---------------+--------------+------+------------+--------------+------------+------------------------------+------+----------------+--------------+-------------------+-------------------+------------+------------------+------------+--------------+------------------+---------------+-------------------+-----------------------+--------------+----------------+-----------------------+------+-----------------+-------------------+------------+----------+--------------------------+---------------+------------------+------------+--------+------+-----------+------------+------------+-------------+-------------+--------------+-------------+--------------+--------

In [38]:
spark.sql("CREATE NAMESPACE nessie.oms").show()

++
||
++
++



In [54]:
# spark.sql("DROP NAMESPACE nessie.test").show()

++
||
++
++



In [39]:
# spark = SparkSession.builder.config(conf=conf).getOrCreate()
# print("Spark Session Started")
df.writeTo("nessie.oms.oms_data_raw").createOrReplace()

25/02/27 17:51:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk
 Schema: Createdon, punchCloseAt, OutageType, saifi, saidi, consumer_count_fdr5, outageSubType, initialoffreason, htclosingreason, fdr_Id, fdr_name, grid_name, Relay, isemergency, israintripping, TAT, duration, dts_Id, consumer_count_fdr18, datetime, tempmax, tempmin, temp, dew, humidity, precip, precipprob, precipcover, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solare

In [101]:
spark.read.table("nessie.test.test_data_raw").show()


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Createdon,punchCloseAt,OutageType,saifi,saidi,consumer_count_fdr,outageSubType,initialoffreason,htclosingreason,fdr_Id,fdr_name,grid_name,Relay,isemergency,israintripping,TAT,duration,dts_Id,consumer_count_fdr,datetime,tempmax,tempmin,temp,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk|
+---------------------------------------------------------------------------------------------------------------------------------------------------------

In [102]:
spark.sql("SELECT * from nessie.fact_billing_and_recovery_final1.fact_billing_and_recovery_data_raw LIMIT 10;").show()

+-------+-------------------+-------------------+----------+------------+----------+---------------+------------------+--------------------+------------+--------------------+-----------------+--------------+----------------+
|Fact_ID|Billing_Document_ID|Contract_Account_ID|  Due_Date|Total_Amount|Tax_Amount|Recovery_Amount|Outstanding_Amount|Average_Payment_Time|Units_Billed|Benefits_Adjustments|Import_Tax_Amount|Energy_Charges|Surcharge_Amount|
+-------+-------------------+-------------------+----------+------------+----------+---------------+------------------+--------------------+------------+--------------------+-----------------+--------------+----------------+
|      0|       100056932870|       400000000009|22.01.2024|   163068.55|      NULL|   2.72257278E8|   -2.7209420945E8|                NULL|      2538.0|             4697.81|              0.0|      91680.96|          2538.0|
|      1|       100056953384|       400000000009|23.01.2024|   161306.28|      NULL|   2.72257278E8|

In [9]:
# Load data into DataFrames
billing_data = spark.read.table("nessie.billing.billing_data_raw")
fault_tickets = spark.read.table("nessie.fault_tickets.fault_tickets_data_raw")
energy_consumption = spark.read.table("nessie.EnergyConsumptionFeederwise.EnergyConsumptionFeederwise_data_raw")
feeder_master = spark.read.table("nessie.feedermaster.feedermaster_data_raw")
feeder_voltage = spark.read.table("nessie.feedervoltage.feedervoltage_data_raw")
pmt_master = spark.read.table("nessie.pmtmaster.pmtmaster_data_raw")
power_report = spark.read.table("nessie.power_report.power_report_data_raw")
recovery_data = spark.read.table("nessie.recovery.recovery_data_raw")

# Register temporary views
energy_consumption.createOrReplaceTempView("energy_consumption")
billing_data.createOrReplaceTempView("billing_data")
fault_tickets.createOrReplaceTempView("fault_tickets")
feeder_master.createOrReplaceTempView("feeder_master")
feeder_voltage.createOrReplaceTempView("feeder_voltage")
pmt_master.createOrReplaceTempView("pmt_master")
power_report.createOrReplaceTempView("power_report")
recovery_data.createOrReplaceTempView("recovery_data")


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


## Testing

In [10]:
df_heavy_join = spark.sql("""
SELECT 
    r.`Overall Total`,
    b.`LPS Billed`, 
    (r.`Overall Total` + b.`LPS Billed`) * 100 AS Recovery_Percentage
FROM recovery_data r
INNER JOIN billing_data b 
ON r.`Posting date in the document` = b.`Posting date in the document`;

""")
df_heavy_join.show(10)  # Action: Triggers execution
# from pyspark.sql.functions import col

# numPartitions = 500
# recovery_data = recovery_data.repartition(numPartitions)
# billing_data = billing_data.repartition(numPartitions)


# # Repartition both tables on "IBC" for better join performance
# recovery_data = recovery_data.repartition("IBC")
# billing_data = billing_data.repartition("IBC")

# # Select only required columns to reduce memory usage
# recovery_data = recovery_data.select("IBC", "Posting date in the document", "Overall Total")
# billing_data = billing_data.select("IBC", "Posting date in the document", "LPS Billed")

# # Perform the FULL OUTER JOIN
# result = recovery_data.alias("r").join(
#     billing_data.alias("b"),
#     on=["IBC", "Posting date in the document"],
#     how="full_outer"
# ).select(
#     col("Overall Total"),
#     col("LPS Billed"),
#     (col("Overall Total") + col("LPS Billed") * 100).alias("Recovery_Percentage")
# )

# # Cache the result for repeated use
# result.cache()

# # Show the first 10 rows
# result.show(10)


+-------------+----------+-------------------+
|Overall Total|LPS Billed|Recovery_Percentage|
+-------------+----------+-------------------+
|     200000.0|    788.46|        2.0078846E7|
|     200000.0|    443.67|        2.0044367E7|
|     200000.0|   -260.87|        1.9973913E7|
|     200000.0|   -426.19|        1.9957381E7|
|     200000.0|   -124.58|        1.9987542E7|
|     200000.0|       0.0|              2.0E7|
|     200000.0|       0.0|              2.0E7|
|     200000.0|    763.56|        2.0076356E7|
|     200000.0|     259.2|         2.002592E7|
|     200000.0|     260.0|           2.0026E7|
+-------------+----------+-------------------+
only showing top 10 rows



In [ ]:
spark.sql("SHOW TABLES IN nessie").show()

In [7]:
spark.sql("DESCRIBE power_report").show(truncate=False)

+---------------------+---------+-------+
|col_name             |data_type|comment|
+---------------------+---------+-------+
|Time                 |timestamp|NULL   |
|Feeder               |int      |NULL   |
|Device ID            |string   |NULL   |
|DTS ID               |string   |NULL   |
|Active Power (kW)    |string   |NULL   |
|Reactive Power (kVAR)|string   |NULL   |
|Apparent Power (kVA) |string   |NULL   |
+---------------------+---------+-------+



In [8]:
spark.sql("DESCRIBE fault_tickets").show(truncate=False)

+-----------------+---------+-------+
|col_name         |data_type|comment|
+-----------------+---------+-------+
|Ticket No        |bigint   |NULL   |
|Ticket Created at|string   |NULL   |
|TicketDTSID      |string   |NULL   |
|Feeder ID        |string   |NULL   |
|Noti. No         |string   |NULL   |
|UserStatus       |string   |NULL   |
|SystemStatus     |string   |NULL   |
|Notification Time|string   |NULL   |
|Emergency Type   |string   |NULL   |
|CompletedAt      |string   |NULL   |
|Subject          |string   |NULL   |
|AreaSecuredTime  |string   |NULL   |
|TrueCallerTime   |string   |NULL   |
|Item Description |string   |NULL   |
|IsAreaSecured    |string   |NULL   |
|AreaSecureTAT    |string   |NULL   |
|IsTrueCaller     |string   |NULL   |
|TrueCallerTAT    |string   |NULL   |
|CreatedBy        |string   |NULL   |
+-----------------+---------+-------+



In [9]:
query = """
CREATE OR REPLACE TEMP VIEW fact_network_and_losses AS
SELECT
    -- Keys
    fm.Feeder AS Feeder_ID,  -- Using `Feeder` from feeder_master
    ec.`Device ID` AS Device_ID,  -- Corrected to use `Device ID` with backticks
    pm.`DTS ID` AS Grid_Station_ID,  -- Corrected to use `DTS ID` with backticks
    ft.`Ticket No` AS Ticket_No,
    
    -- Metrics
    COUNT(ft.Subject) AS Fault_Count,
    SUM(CASE WHEN ft.SystemStatus = 'UnPlanned' THEN 1 ELSE 0 END) +
    SUM(ppr.`Active Power (kW)`) AS Energy_Created,  -- Changed to `ppr` as column exists there
    SUM(ppr.`Reactive Power (kVAR)`) AS Energy_Transmitted,  -- Changed to `ppr` as column exists there
    COUNT(ft.`Emergency Type`) AS Emergency_Count,
    COUNT(ft.`Ticket No`) AS Tickets_Processed,
    AVG(`ft`.`CompletedAt` - ft.`Notification Time`) AS Fault_Resolution_Time,
    MAX(ft.Subject) AS Most_Common_Fault
FROM
    fault_tickets ft
    INNER JOIN feeder_master fm ON ft.`Feeder ID` = fm.Feeder
    INNER JOIN energy_consumption ec ON fm.Feeder = ec.FeederID
    INNER JOIN pmt_master pm ON pm.`DTS ID` = ft.`TicketDTSID`
    INNER JOIN power_report ppr ON ppr.`Device ID` = ec.`Device ID`
GROUP BY
    fm.Feeder, ec.`Device ID`, pm.`DTS ID`, ft.`Ticket No`;
"""

# Execute the query
spark.sql(query)


DataFrame[]

In [10]:
spark.sql("SELECT * FROM billing_data").show(10)

+----------------+-------------+------------+----------------------+------------------+--------------+------------+--------------+--------------+-------------------------------------------+----------------------------+------------------+--------------------------------------+----------------+------------+-------------------+-----------------------------------+---------------+---------------------------------------+----------+-------------+-----------------------+----------------+------+---------+----------------------------+-----------------+------+-------------------+---------+--------------------+---------------+-------------------------------------+-------------------+--------------+------------------+---------------+----------------+-------------------+---------------------+--------------------+------------+--------------------+--------------------+----------------+--------------+-------------------+-------------------+-------------+----------------+-----------------+------------+-

In [128]:
query2 = """
CREATE OR REPLACE TEMP VIEW fact_billing_and_recovery AS
SELECT
    -- Primary Key
    MONOTONICALLY_INCREASING_ID() AS Fact_ID, -- Unique identifier for each record in the fact table
    
    -- Foreign Keys
    b.`Billing Document` AS Billing_Document_ID, -- Foreign key to billing_data
    r.`Contract Account (Partner-Independent Data)` AS Contract_Account_ID, -- Foreign key to recovery_data
    
    -- Additional Date Fields for Analysis
    b.`Due Date (Print Doc)` AS Due_Date, -- Correct column with date conversion
    
    -- Metrics
    SUM(b.`Net Amount`) AS Total_Amount, -- Summation of Net Amount for Total_Amount
    SUM(
        b.Tariff +
        b.GST +
        b.`GST (Further 1%)` +
        b.`GST (Extra 5%)` +
        b.`GST_Retailer` +
        b.`GST on Retailers (5 %)` +
        b.`GST on Retailers (7.5 %)` +
        b.`GST` +
        b.`Income Tax` +
        b.`Income Tax Surcharge` +
        b.`Normal GST on import` +
        b.`WithHolding GST -IMP` +
        b.`WithHolding ITax-IMP` +
        b.Surcharge +
        b.`Surcharge_1` +
        b.`GST on Marble` +
        b.`Electricity Duty` +
        b.`Additional Surcharge` +
        b.`MUCT (KMC)`
    ) AS Tax_Amount, -- Summation of all specified tax components
    SUM(r.`Total Amount`) AS Recovery_Amount, -- Total recovery amount from recovery
    (SUM(b.`Net Amount`) - SUM(r.`Total Amount`)) AS Outstanding_Amount, -- Difference between Total Amount and Recovery Amount
    AVG(DATEDIFF(r.`Document Date`, b.`Issue Date (Print Doc's Posting Date)`)) AS Average_Payment_Time, -- Document Date - Issue Date for payment time
    SUM(b.`Units Billed`) AS Units_Billed, -- Total units billed
    SUM(
        b.`Emp Benefit -Officer` +
        b.`Free Benefit Amount` +
        b.`PM Relief` +
        b.`PM Relief Res. Comm.` +
        b.`Award Winner Benefit` +
        b.`ISP Adj.` +
        b.`Fuel Adj. Charges` +
        b.`Sales Tax Adjustment` +
        b.`Uniform Qtr.Adj. on Import` +
        b.`Deferred FCA Adj. Im` +
        b.`Deferred FCA Adjustm` +
        b.`ISPA Adj. Ind. Peak` +
        b.`Quarterly Adj.` +
        b.`Uniform Quatrly Adj.` +
        b.`New Qtr Adj Import Off Peak` +
        b.`New Qtr Adj Import Peak` +
        b.`New Qtr Adj Export Off Peak` +
        b.`New Qtr Adj Export Peak` +
        b.`Quarterly Adj. Curre`
    ) AS Benefits_Adjustments, -- Summation of benefits and adjustments
    SUM(
        b.`Import-OffPeak Units` +
        b.`Import-On Peak Units` +
        b.`Import Units Off Peak – Current` +
        b.`Import Units Non-TOU – Current` +
        b.`Consp-Non TOD-Import`
    ) AS Import_Tax_Amount, -- Summation of import-related taxes
    SUM(b.`Energy Charges`) AS Energy_Charges, -- Energy charges from billing
    SUM(b.`Units Billed`) AS Surcharge_Amount -- Replacing with summation of units billed
FROM
    billing_data b
    INNER JOIN recovery_data r 
    ON b.`Contract Account (Partner-Independent Data)` = r.`Contract Account (Partner-Independent Data)`
WHERE
    b.`Contract Account (Partner-Independent Data)` IS NOT NULL -- Exclude rows with NULL Contract Account
GROUP BY
    b.`Billing Document`,
    r.`Contract Account (Partner-Independent Data)`,
    b.`Due Date (Print Doc)`;
"""

spark.sql(query2)
print("hakuna matata, table banata")


hakuna matata, table banata


In [130]:
spark.sql("SELECT * FROM fact_billing_and_recovery").show(10)

+-------+-------------------+-------------------+----------+------------+----------+---------------+------------------+--------------------+------------+--------------------+-----------------+--------------+----------------+
|Fact_ID|Billing_Document_ID|Contract_Account_ID|  Due_Date|Total_Amount|Tax_Amount|Recovery_Amount|Outstanding_Amount|Average_Payment_Time|Units_Billed|Benefits_Adjustments|Import_Tax_Amount|Energy_Charges|Surcharge_Amount|
+-------+-------------------+-------------------+----------+------------+----------+---------------+------------------+--------------------+------------+--------------------+-----------------+--------------+----------------+
|      0|       100056932870|       400000000009|22.01.2024|   163068.55|      NULL|   2.72257278E8|   -2.7209420945E8|                NULL|      2538.0|             4697.81|              0.0|      91680.96|          2538.0|
|      1|       100056934751|       400000000009|22.01.2024|   171915.17|      NULL|   2.72257278E8|

In [14]:
# spark.sql("CREATE NAMESPACE nessie.fact_network_and_loss;").show()
spark.sql("CREATE NAMESPACE nessie.fact_billing_and_recovery_final1;").show()

++
||
++
++



In [11]:
spark.sql("CREATE NAMESPACE nessie.fact_network_and_losses;").show()

++
||
++
++



In [ ]:
fact_network_and_loss_df = spark.sql("SELECT * FROM fact_network_and_losses")

fact_network_and_loss_df.writeTo("nessie.fact_network_and_losses.fact_network_and_loss_data_raw").createOrReplace()

print ("minio shinio here we go!!")

In [126]:
# Assuming you have already created a DataFrame `fact_billing_and_recovery`
fact_billing_and_recovery_df = spark.sql("SELECT * FROM fact_billing_and_recovery")

# Writing the DataFrame into Dremio
fact_billing_and_recovery_df.writeTo("nessie.fact_billing_and_recovery_final1.fact_billing_and_recovery_data_raw").createOrReplace()

print("dou piyasi in dremio lesso (fact_table for billing shilling)")


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `fact_billing_and_recovery` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [fact_billing_and_recovery], [], false


In [23]:
spark.read.table("nessie.fact_network_and_loss2.fact_network_data_raw").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `nessie`.`fact_network_and_loss2`.`fact_network_data_raw` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [nessie, fact_network_and_loss2, fact_network_data_raw], [], false


### Appending values

In [95]:
spark.stop()

In [5]:
spark.sql("""
    SELECT *
    FROM nessie.fact_network_and_loss11.fact_network_and_loss_data_raw.snapshots
""").show(truncate=False)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `nessie`.`fact_network_and_loss11`.`fact_network_and_loss_data_raw`.`snapshots` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Project [*]
+- 'UnresolvedRelation [nessie, fact_network_and_loss11, fact_network_and_loss_data_raw, snapshots], [], false
